In [ ]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai


async def get_opening_line(scraped_content: str) -> str:
    # Assuming you have OpenAI API setup
    user_prompt = f"""Based on this website content, create a friendly opening line for a customer service agent:
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating opening lines for customer service agents. You are given a website content and you are tasked with creating a friendly opening line for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agents_from_urls(urls: List[str], instr_agent: str):
    for url in urls:
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        
        # Map and scrape the URL
        mapped_url = await map_url(url)
        scraped_content = await scrape_url(mapped_url)
        
        # Generate opening line from content
        opening_line = await get_opening_line(scraped_content)
        
        # Create data source dict
        data_source = [{
            "id": str(uuid.uuid4()),
            "title": url,
            "data_type": "web"
        }]
        
        # Prepare payload
        payload = {
            "agentName": agent_name,
            "agentPurpose": f"Customer service agent for {agent_name}",
            "dataSource": data_source,
            "openingLine": opening_line,
            "language": "en-GB", 
            "voice": "Alex K",
            "voiceProvider": "cartesia",
            "instructions": instr_agent,

        }
        
        # Send POST request
        response = requests.post(
            "http://localhost:8000/api/v1/livekit/new_agent",
            json=payload,
            headers={"x-user-id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"}
        )
        
        if response.status_code != 200:
            print(f"Failed to create agent for {url}: {response.text}")
        else:
            print(f"Successfully created agent for {url}")

# Example usage:
urls = [
    "https://example1.com"
]
instr_agent = "Be helpful and friendly when assisting customers."

await create_agents_from_urls(urls, instr_agent)